# 7. Folium

- Pin all launch sites on a map
- Pin all success/failed launches for each site on the map
- Find distances from launch sites to their nearest coastline

## TOC

- [Maps Setup](#maps-setup)
- [Helpers](#helpers)
- [Load data](#load-data)
- [Data Wrangling](#data-wrangling)
- [Launch sites](#plot-launch-sites)
- [Success/failure rate](#plot-successfailure-rate)
- [Distances to coastlines](#plot-distances-to-coastlines)


## Imports


In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon
from pathlib import Path

from math import sin, cos, sqrt, atan2, radians

import helpers as hlp

## Maps setup


In [2]:
INPUT_FILE = hlp.DATA_DIR / Path("spacex_launch_geo.csv")

# Folium settings
MAP_WIDTH = 900
MAP_HEIGHT = 500
MAP_TILES = "Cartodb Positron"  # "OpenStreetMap"
MAP_LOCATION = [30, -99]
MAP_ZOOM = 4
MAP_LABEL_SIZE = "1.1rem"

TITLE_COLOR = "#2277ff"
TITLE_CSS = f"""
display: flex;
justify-content: center;
width: 100%; 
background-color: {TITLE_COLOR}; 
color: white;
line-height: 2em; 
font-weight: bold;
"""

## Helpers


In [3]:
def calculate_distance(coords_1: list[float], coords_2: list[float]):
    """Return the distance between 2 pairs of coordinates"""
    # approximate radius of earth in km
    R = 6373.0

    lat1, lon1 = coords_1
    lat2, lon2 = coords_2
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)

## Load data


In [4]:
df = pd.read_csv(INPUT_FILE, encoding="utf-8", header=0, index_col=False)

# normalize column names
df.columns = [hlp.normalize_column_name(col) for col in df.columns]
df.head()

,flight_number,date,time_utc,booster_version,launch_site,payload,payload_mass_kg,orbit,customer,landing_outcome,class,lat,long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [5]:
df["booster_version"].value_counts()

booster_version
F9 v1.1           5
F9 v1.0  B0003    1
F9 v1.0  B0005    1
F9 v1.0  B0004    1
F9 v1.0  B0006    1
F9 v1.0  B0007    1
F9 v1.1 B1011     1
F9 v1.1 B1010     1
F9 v1.1 B1012     1
F9 v1.1 B1013     1
F9 v1.1 B1014     1
F9 v1.1 B1015     1
F9 v1.1 B1016     1
F9 v1.1 B1018     1
F9 FT B1019       1
F9 FT B1020       1
F9 FT B1021.1     1
F9 FT B1022       1
F9 FT B1023.1     1
F9 FT B1024       1
F9 FT B1025.1     1
F9 FT B1026       1
F9 v1.1  B1003    1
F9 v1.1 B1017     1
F9 FT B1029.1     1
F9 FT B1036.1     1
F9 FT B1038.1     1
F9 B4 B1041.1     1
F9 FT  B1036.2    1
F9 FT  B1038.2    1
F9 B4  B1041.2    1
F9 B4  B1043.2    1
F9 FT B1031.1     1
F9 FT B1030       1
F9 FT  B1021.2    1
F9 FT B1032.1     1
F9 FT B1034       1
F9 FT B1035.1     1
F9 FT  B1029.2    1
F9 FT B1037       1
F9 B4 B1039.1     1
F9 B4 B1040.1     1
F9 FT  B1031.2    1
F9 B4 B1042.1     1
F9 B5  B1046.1    1
F9 FT  B1035.2    1
F9 B4 B1043.1     1
F9 FT  B1032.2    1
F9 B4 B1044       1
F9 B

## Data Wrangling


In [6]:
coastline_coords = {
    "CCAFS LC-40": (28.5623, -80.56789),
    "CCAFS SLC-40": (28.5631, -80.56789),
    "KSC LC-39A": (28.57269, -80.60626),
    "VAFB SLC-4E": (34.633, -120.62735),
}
city_coords = {
    "CCAFS LC-40": (28.614, -80.849),
    "CCAFS SLC-40": (28.614, -80.849),
    "KSC LC-39A": (28.614, -80.849),
    "VAFB SLC-4E": (34.646, -120.468),
}

df = df.rename(columns={"long": "lon"})
# add columns for later use
df["marker_color"] = df["class"].apply(
    lambda outcome: "green" if outcome == 1 else "red"
)

# add coastline coordinates to dataframe
for location in coastline_coords:
    lat, lon = coastline_coords[location]
    df.loc[df["launch_site"] == location, "coastline_lat"] = lat
    df.loc[df["launch_site"] == location, "coastline_lon"] = lon
# add city coordinates to dataframe
for location in city_coords:
    lat, lon = city_coords[location]
    df.loc[df["launch_site"] == location, "city_lat"] = lat
    df.loc[df["launch_site"] == location, "city_lon"] = lon

df["coastline_distance"] = df.apply(
    lambda row: calculate_distance(
        [row["lat"], row["lon"]], [row["coastline_lat"], row["coastline_lon"]]
    ),
    axis=1,
)
df["city_distance"] = df.apply(
    lambda row: calculate_distance(
        [row["lat"], row["lon"]], [row["city_lat"], row["city_lon"]]
    ),
    axis=1,
)

df["lat"] = df["lat"].round(3)
df["lon"] = df["lon"].round(3)

df.head()

,flight_number,date,time_utc,booster_version,launch_site,payload,payload_mass_kg,orbit,customer,landing_outcome,class,lat,lon,marker_color,coastline_lat,coastline_lon,city_lat,city_lon,coastline_distance,city_distance
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15


In [7]:
df_temp = df[["launch_site", "lat", "lon", "class", "marker_color"]]
df_locations = df_temp.groupby(["launch_site"], as_index=False)[
    ["launch_site", "lon", "lat"]
]
df_locations = df_locations.value_counts()[["launch_site", "lat", "lon"]]

In [8]:
df_locations

,launch_site,lat,lon
0,CCAFS LC-40,28.562,-80.577
1,CCAFS SLC-40,28.563,-80.577
2,KSC LC-39A,28.573,-80.647
3,VAFB SLC-4E,34.633,-120.611


## Misc mapping functions


In [9]:
def build_map() -> folium.Map:
    """HELPER.
    Return a folium.Map object, given a series of defaults defined earlier."""
    coords_formatter = "function(num) {return L.Util.formatNum(num, 3);};"
    mouse_position = MousePosition(
        position="topright",
        separator=" Lon: ",
        empty_string="NaN",
        lng_first=False,
        num_digits=20,
        prefix="Lat:",
        lat_formatter=coords_formatter,
        lng_formatter=coords_formatter,
    )
    site_map = folium.Map(
        location=MAP_LOCATION,
        tiles=MAP_TILES,
        zoom_start=MAP_ZOOM,
        zoom_control=True,
    )
    site_map.add_child(mouse_position)
    return site_map


def build_title(title: str) -> folium.Element:
    """HELPER.
    Return a folium.Element object with a given text as a title."""
    return folium.Element(
        f"""<div>
            <h3 style='{TITLE_CSS}'>{title}</h3>
        </div>"""
    )


def build_distance_line(coords_1, coords_2, color: str = "red") -> folium.PolyLine:
    """HELPER.
    Return a folium.PolyLine object between 2 sets of coordinates in a given color."""
    return folium.PolyLine(
        color=color,
        dash_array="10",
        locations=[coords_1, coords_2],
        smooth_actor=4,
        weight=4,
    )


def build_distance_marker(
    coords_1, coords_2, distance, color: str = "red"
) -> folium.Marker:
    """HELPER.
    Return a Marker object placed between 2 sets of coordinates and in a given color."""
    icon = folium.DivIcon(
        html=f"<div style='font-size: {MAP_LABEL_SIZE}; color: {color}'>{distance:.2f} Km.</div>",
        icon_size=(250, 40),
        icon_anchor=(20, 20),
    )
    marker = folium.Marker(location=calc_mid_position(coords_1, coords_2))
    marker.add_child(icon)
    return marker


def build_site_marker(name: str, coords: list, color: str = "blue"):
    """HELPER.
    Return a folium.Marker object containing the name of the launch site."""
    icon = folium.DivIcon(
        html=f"<div style='font-size: {MAP_LABEL_SIZE}; color: blue'>{name}</div>",
        icon_size=(250, 40),
        icon_anchor=(20, 20),
    )
    marker = folium.Marker(location=[coords[0] - 0.03, coords[1]])
    marker.add_child(icon)
    return marker


def calc_mid_position(coords_1, coords_2):
    """Given 2 sets of coordinates, each divided into x and y, or latitude and longitude, return the middle point.
    Helps positioning titles in between 2 points on a map."""
    return [
        (coords_1[0] + coords_2[0]) / 2,
        (coords_1[1] + coords_2[1]) / 2,
    ]

## Plot Launch sites


In [10]:
def graph_launch_sites(locations: pd.DataFrame):
    title = build_title("Falcon 9 launch locations")
    fig = folium.Figure(width=MAP_WIDTH, height=MAP_HEIGHT)
    site_map = build_map()
    fig.add_child(site_map)

    for _, row in locations.iterrows():
        site_name, lat, long = row.values
        html = f'<div style="font-size:18;color:#000;"><b>{site_name}</b></div>'
        coords = [lat, long]
        circle = folium.Circle(coords, radius=1000, color="red", fill=True)
        icon = folium.Icon(
            color="red",
            icon="rocket",
            prefix="fa",
            icon_size=(35, 45),
            icon_anchor=(20, 50),
            shadow_size=0,
        )
        marker = folium.Marker(
            location=coords,
            popup=site_name,
            icon=DivIcon(icon_size=(35, 45), icon_anchor=(10, -10), html=html),
        )
        marker.add_child(icon)
        site_map.add_child(marker)
        site_map.add_child(circle)

    fig.get_root().html.add_child(title)
    fig.add_child(site_map)
    return fig


site_map = graph_launch_sites(df_locations)
site_map

## Success/Failure instances on each launch site


In [11]:
df.head(3)

,flight_number,date,time_utc,booster_version,launch_site,payload,payload_mass_kg,orbit,customer,landing_outcome,class,lat,lon,marker_color,coastline_lat,coastline_lon,city_lat,city_lon,coastline_distance,city_distance
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15


## PLot success/failure rate


In [12]:
def graph_success_failure_rate(df: pd.DataFrame):
    title = build_title("Success/Failure launches")
    fig = folium.Figure(width=MAP_WIDTH, height=MAP_HEIGHT)
    site_map = build_map()
    fig.add_child(site_map)
    marker_cluster = MarkerCluster()

    for _, row in df.iterrows():
        site_name, lat, lon, classs, marker_color = row.values
        coords = [lat, lon]
        circle = folium.Circle(
            coords,
            radius=800,
            color="red",
            fill=True,
            fill_opacity=0.01,
        )
        marker = folium.Marker(location=coords, popup=site_name)
        icon = folium.Icon(
            color="white", icon_color=marker_color, icon_anchor=(20, 50), shadow_size=0
        )
        marker.add_child(icon)
        marker_cluster.add_child(marker)
        site_map.add_child(circle)
        site_map.add_child(marker_cluster)

    fig.get_root().html.add_child(title)
    fig.add_child(site_map)
    return fig


site_map = graph_success_failure_rate(
    df[["launch_site", "lat", "lon", "class", "marker_color"]]
)
site_map

## Plot distances to coastlines


In [13]:
df.head()

,flight_number,date,time_utc,booster_version,launch_site,payload,payload_mass_kg,orbit,customer,landing_outcome,class,lat,lon,marker_color,coastline_lat,coastline_lon,city_lat,city_lon,coastline_distance,city_distance
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562,-80.577,red,28.5623,-80.56789,28.614,-80.849,0.92,27.15


In [14]:
# def temp_print(df: pd.DataFrame):
#     lines_coords = []
#     i = 0
#     for _, row in df.iterrows():
#         (
#             site_name,
#             lat,
#             lon,
#             coastline_lat,
#             coastline_lon,
#             coastline_distance,
#             city_lan,
#             city_lon,
#             city_distance,
#         ) = row.values
#         site_coords = [lat, lon]
#         coastline_coords = [coastline_lat, coastline_lon]
#         lines_coords.append([site_coords, coastline_coords])
#         # i += 1
#         # if i >= 10:
#         #     break
#     print(lines_coords)


# coords_df = df[
#     [
#         "launch_site",
#         "lat",
#         "lon",
#         "coastline_lat",
#         "coastline_lon",
#         "coastline_distance",
#         "city_lat",
#         "city_lon",
#         "city_distance",
#     ]
# ]
# temp_print(coords_df)

In [15]:
coords_df = pd.DataFrame()
for launch_site in df["launch_site"].unique():
    row = df.loc[(df["launch_site"] == launch_site)][
        [
            "lat",
            "lon",
            "coastline_lat",
            "coastline_lon",
            "coastline_distance",
            "city_lat",
            "city_lon",
            "city_distance",
        ]
    ].head(1)
    lat = row["lat"]
    lon = row["lon"]
    coast_lat = row["coastline_lat"]
    coast_lon = row["coastline_lon"]
    coast_distance = row["coastline_distance"]
    city_lat = row["city_lat"]
    city_lon = row["city_lon"]
    city_distance = row["city_distance"]
    new_site = pd.DataFrame.from_records(
        {
            "launch_site": launch_site,
            "lat": lat,
            "lon": lon,
            "coast_lat": coast_lat,
            "coast_lon": coast_lon,
            "coast_distance": coast_distance,
            "city_lat": city_lat,
            "city_lon": city_lon,
            "city_dist": city_distance,
        }
    )
    coords_df = pd.concat([coords_df, new_site], ignore_index=True)
coords_df

,city_dist,city_lat,city_lon,coast_distance,coast_lat,coast_lon,lat,launch_site,lon
0,27.15,28.614,-80.849,0.92,28.56230,-80.56789,28.562,CCAFS LC-40,-80.577
1,13.15,34.646,-120.468,1.52,34.63300,-120.62735,34.633,VAFB SLC-4E,-120.611
2,20.25,28.614,-80.849,3.97,28.57269,-80.60626,28.573,KSC LC-39A,-80.647
3,27.18,28.614,-80.849,0.87,28.56310,-80.56789,28.563,CCAFS SLC-40,-80.577


In [16]:
def graph_distances_to_coastline_2(df: pd.DataFrame):
    title = build_title("Distances to coastlines & cities")
    fig = folium.Figure(width="800", height="500")
    site_map = build_map()

    # Traverse our dataframe, adding information data points to our map object
    for _, row in df.iterrows():
        # Grab the required data
        site_coords = [row["lat"], row["lon"]]
        coast_coords = [row["coast_lat"], row["coast_lon"]]
        coast_dist = row["coast_distance"]
        city_coords = [row["city_lat"], row["city_lon"]]
        city_dist = row["city_dist"]
        launch_site = row["launch_site"]

        # site center
        circle = folium.Circle(site_coords, radius=8000, color=TITLE_COLOR, fill=False)

        # launch site name
        launch_site_marker = build_site_marker(
            launch_site.split()[0], site_coords, "blue"
        )

        # create a line to the nearest coastline
        coast_line = build_distance_line(site_coords, coast_coords, "red")
        # add the distance to the coastline
        coast_dist_marker = build_distance_marker(
            site_coords, coast_coords, coast_dist, "red"
        )

        # create a line to the nearest city
        city_line = build_distance_line(site_coords, city_coords, "blue")
        # add the distance to the city
        city_dist_marker = build_distance_marker(
            site_coords, city_coords, city_dist, "blue"
        )

        # Add everything to the map
        site_map.add_child(launch_site_marker)
        site_map.add_child(circle)
        site_map.add_child(coast_dist_marker)
        site_map.add_child(coast_line)
        site_map.add_child(city_dist_marker)
        site_map.add_child(city_line)

    fig.add_child(site_map)
    fig.get_root().html.add_child(title)
    return fig


site_map = graph_distances_to_coastline_2(coords_df)
site_map